[toc]


# 大数据算法之蓄水池算法

蓄水池算法解决的是流数据抽样的问题。我们先来解决固定总体的抽样，再来解决流数据的抽样问题。

## 什么是均匀抽样？

对于什么是“均匀抽样”。个人目前有两种理解。
1. 每个元素最终出现在最终结果中的可能性都相等，是 $\frac{k}{n}$
2. 每个样本组合出现的可能都相等，是 $C_n^k$，如当 k = 3, n = 5 时， A[1], A[2], A[3] 是一个可能的样本组合，A[1], A[3], A[4] 也是一个可能的样本组合。他们出现的概率均为 $C_5^3$。

从直观上看，我们所谓的均匀抽样应该是第 2 种理解。而第 1 种理解下的“均匀”成立的条件下，第 2 种理解下的“均匀”必然成立。但是第 2 种理解下的均匀成立时，第 1 种理解下的“均匀”是否成立笔者还不清楚。

## 固定总体抽取 k 个样本

问题：从给定长度为 n 的数组中随机抽取 k 个样本，要求均匀抽样，如何实现？

### 思路1：

实现这个问题，最简单的方式就是抽取 [0, k] 之间的随机数，抽取 k 个，然后返回对应下标的数即可。如果随机抽取出来的下标有重复的怎么办？只需要将重复的数丢弃重新抽取就可以了，代码如下：

In [1]:
import random

A = [10, 20, 30, 40, 50, 60]
n = len(A) 
k = 2

def randomSample(A, n, k):
    indices = set()
    while len(indices) < k:
        random_idx = random.randint(0, n-1)
        indices.add(random_idx)
    return [A[idx] for idx in indices]

print(randomSample(A, n, k))

[30, 60]


这种做法的正确性是显然的。

### 思路2: 利用 RANDOMIZE-IN-PLACE 算法

In [ ]:
我们想要均匀抽取 k 个元素。我们可以直接将整个数组 shuffle 之后取前 k 个元素即可。只要数组的 shuffle 是均匀的，我们的抽样也是均匀的。我们可以利用 RANDOMIZE-IN-PLACE 算法来进行均匀 shuffle。

进一步，我们甚至不需要 shuffle 整个数组， 只需要在 RANDOMIZE-IN-PLACE 的第 k 个循环处暂停即可。

因为RANDOMIZE-IN-PLACE 算法的不变式是：
当第 k 个循环完成时，数组的前 k 个元素是原数组的一个 k 排列。

利用这个性质，我们无需 shuffle 整个数组就可以实现从固定总体中均匀抽取 k 个样本的问题。

In [25]:
import random

A = [10, 20, 30, 40, 50, 60]
n = len(A) 
k = 2

def randomSample(A, n, k):
    i = 0
    for i in range(k):
        j = random.randint(i, n-1) # random sample from [i, n-1]
        A[i], A[j] = A[j], A[i]
    return A[:k]

print(randomSample(A, n, k))

[60, 40]


## 不固定总体抽取 k 个元素 —— 流数据抽样

问题：从不定长度的数据流中随机抽取 k 个样本，要求均匀抽样，如何实现？

我们先给出算法，再证明，最后给出一些直观思路。

### 算法：蓄水池算法

假设当前获取的元素是从数据流中获取的第 n 个元素，

In [ ]:
if n <= k:
    保留当前元素
else:
    以 (n-k)/n 的概率随机丢弃掉当前元素。如果当前元素被丢弃，那么继续向下进行；如果当前元素被保留，那么就从之前保留的 k 个元素中随机选择一个元素丢弃，相当于用当前元素随机替换掉了之前保留的某个元素。

### 算法正确性证明

假设第 n 步，保留的 k 个元素分别为 x1, ..., xk

要证明算法正确性，只需证明，$p(x1,..,xk) = \frac{1}{c^k_n}$

使用数学归纳法

1. n == k 时，只有一种抽样方式，因此这个样本被抽取的概率为 1 (n < k 时没有 k 个元素，因此不考虑)
2. n > k 时，由归纳假设，$p(x1,...,xk) = \frac{1}{c^k_n}$
3. n + 1 时，可以将所有可能的结果划分为两类：一类是含 $x_{n+1}$ 的结果，一类是不含 $x_{n+1}$ 的结果

对于不含 $x_{n+1}$ 的结果，有
$$
\begin{aligned}
    p(x1,...,xk) &= p(x1,...,xk|xn被丢弃)p(xn被丢弃) + p(x1,...,xk|xn被选中)p(xn被选中) \\
    &= p(x1,...,xk|xn被丢弃)p(xn被丢弃) \\
    &= p(x1,...,xk) p(xn被丢弃) \\
    &= \frac{1}{c^k_n} \frac{n+1-k}{n+1} \\
    &= \frac{1}{\frac{n!}{(n-k)! k!} \frac{n+1}{n+1-k}} \\
    &= \frac{1}{c^k_{n+1}}
\end{aligned}
$$


对于含 $x_{n+1}$ 的结果，$x_1, \cdots, x_k$ 中必定有一个数被丢弃。不失一般性，设 $x_k$ 被丢弃了。因此，

$$
\begin{aligned}
p(x_1, \cdots, x_{k-1}, x_{n}) &= p(x_1, \cdots, x_{k-1}|x_n被保留) p(x_n被保留) + p(x_1, \cdots, x_{k-1}|x_n被丢弃) p(x_n被丢弃)  \\
&= p(x_1, \cdots, x_{k-1}) p(x_n被保留) + 0\\
&= \frac{1}{c^{k-1}_n} \frac{k}{n+1} \\
&= \frac{1}{\frac{n!}{(n-k+1)!(k-1)!} \frac{n+1}{k}} \\
&= \frac{1}{c^{k}_{n+1}}
\end{aligned}
$$

因此，当从数据流中获取了 n 个元素时，不管哪类结果，这个结果被抽取的概率都是 $\frac{1}{c_n^k}$

### 另一种算法正确性的证明

在网上搜集资料的过程中，实际上不是用上面的证明来论证蓄水池算法的正确性的。而是用另一种方法。这种方法个人认为是基于对均匀抽样的第1种理解得来的。下面也加以叙述：


为了证明算法的正确性，只需要证明，当获取第 n 个元素后，利用本算法得到的每个元素的保留概率都是 $\frac{k}{n}$

1. n == k是，每个元素都被保留，因此保留概率为 $\frac{k}{k} = 1$ 满足条件。
2. 假设 n > k 是，$\forall i \in {1, 2, \cdots, n}, p(x_i 被保留) = \frac{k}{n}$
3. n+1 时，也分两类去计算概率，一类是 $x_{n+1}$, 一类是 $x_{1}, \cdots, x_{n}$。

第一类：
$$
\begin{aligned}
p(x_{n+1}) &= \frac{k}{n+1} \\
\end{aligned}
$$

第二类：
$\forall i \in {1, \cdots, n}$ 有

$$
\begin{aligned}
p(x_i) &= p(x_i | x_{n+1} 被保留) p(x_{n+1} 被保留) + p(x_i | x_{n+1} 被丢弃) p(x_{n+1} 被丢弃) \\
&= p(x_i | x_{n+1} 被保留) p(x_{n+1} 被保留) + p(x_i) p(x_{n+1} 被丢弃) \\
&= p(x_i) p(x_i 没有被丢弃 | x_i) p(x_{n+1} 被保留) + p(x_i) p(x_{n+1} 被丢弃) \\
&= \frac{k}{n} \frac{k-1}{k} \frac{k}{n+1} + \frac{k}{n} \frac{n+1-k}{n+1} \\
&= \frac{k}{n+1}
\end{aligned} 
$$



### 算法直观理解

这个算法如果是之前没有接触过的话，很难想到。但是还是有一些规律可以帮助理解。比如：为什么在获取到第 n 个元素的时候要以 $\frac{k}{n}$ 的概率去保留这个元素？ 

我们可以从第二个证明中得到一点启发。第二个证明中，对于第一类，$p(x_{n+1})$ 直接就等于 $\frac{k}{n+1}$，也就是说，我们这样设置是为了让新获取的元素 $x_{n+1}$ 被保留的概率为 $\frac{k}{n+1}$

### 代码实现

代码的实现十分直观。

In [ ]:
import random

A = [10, 20, 30, 40, 50, 60]
k = 2

def randomSample(A, k):
    i = 0
    ret = 0
    while True:
        try:
            x = A.get()
            i += 1
        except:
            return ret
        if i < k:
            ret.append(x)
        else:
            if random.rand(1, i) <= k: # 
                j = random.rand(k)
                ret[j] = x

注意，这个函数和之前的函数不同，A 相当于一个队列，这里使用 get 方法从队列中获取数据。如果没有数据，就返回。

# References
1. [朋友圈“锦鲤”盛行，如何抽取“锦鲤”？](https://mp.weixin.qq.com/s?src=11&timestamp=1597030487&ver=2513&signature=omE40yo9SlGO0crjFf8-f1EyWEWvh5WibbpLT1btcp6hTf7c3pF8Z26h449tGTI7QMhWzyM9VasPubKLG7KqHUgjdZ6C-yToVkxDJuFAHoKDsr63xMClzN*GWuEvrKu6&new=1)
2. ["锦鲤"抽奖活动背后的随机抽样算法](https://mp.weixin.qq.com/s?src=11&timestamp=1597039962&ver=2513&signature=iFvwYa4eP3*MKZhrUPZIMqp76VyGnVnFH4Wgg6WBcuFVEuJJB7OUCVjXYHjQ0Q1bzaX5F3cNT7JbzJznXEBoflF7v5BWr8HyOvLjQAnPCGT-18xDht6nOUE4YckyMwOk&new=1)